### This notebook basically does what was done in ***Task 1***, but using the package created on ***Task 2***.

In [1]:
# The imports are being executed only when needed
import pandas as pd
from pprint import pprint

Basic configuration of the notebook

In [2]:
# Configure the settings of pandas so the DataFrames can be better printed
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 400)

# The assets folder, where all files common to the project are saved
assets_folder = '../assets/'

Step 1: read CSV files into pandas DataFrames to better understanding the data

In [3]:
df_bg_map = pd.read_csv(f'{assets_folder}background_mapping.csv')
clean_columns = {c: c.strip() for c in df_bg_map}
df_bg_map.rename(columns=clean_columns, inplace=True)

df_bg_1 = pd.read_csv(f'{assets_folder}project_1_background.csv', index_col=0)
df_bg_2 = pd.read_csv(f'{assets_folder}project_2_background.csv', index_col=0)

df_logs_1 = pd.read_csv(f'{assets_folder}project_1_logs.csv', index_col=0)
df_logs_2 = pd.read_csv(f'{assets_folder}project_2_logs.csv', index_col=0)

Step 2: check the difference between datasets

In [4]:
# Verify the difference between the background df schemas
is_bg_schemas_equal = sorted(list(df_bg_1)) == sorted(list(df_bg_2))
print(f'Background schemas are equal? Answer: {is_bg_schemas_equal}')

Background schemas are equal? Answer: False


In [5]:

bg1_not_in_bg2 = sorted([c for c in df_bg_1 if c not in df_bg_2])
bg2_not_in_bg1 = sorted([c for c in df_bg_2 if c not in df_bg_1])

# Verify that the difference between dfs are the ones in the mapping
prj_2_cols = sorted(df_bg_map['Project 1 Question'].values.tolist())
is_prj_2_cols_mapped = prj_2_cols == bg2_not_in_bg1
prj_1_cols = sorted(df_bg_map['Project 2 Question'].values.tolist())
is_prj_1_cols_mapped = prj_1_cols == bg1_not_in_bg2

# and that the columns in the mapping file are indeed switched
is_all_columns_mapped = is_prj_2_cols_mapped and is_prj_1_cols_mapped
print(f'The columns in the mapping represents all the mismatched columns in background DataFrames: {is_all_columns_mapped}')

The columns in the mapping represents all the mismatched columns in background DataFrames: True


In [6]:
# Confirm that the log DataFrames has the same schema
is_logs_schemas_equal = sorted(list(df_logs_1)) == sorted(list(df_logs_2))
print(f'Logs schemas are equal? Answer: {is_logs_schemas_equal}')

Logs schemas are equal? Answer: True


Step 3: solve problem on column "level2dish_coded"


In [7]:
from sbees.insecure import clean_dict_column

In [8]:
'''For performance reasons, it is necessary to do some cleaning on the
datasets before doing the merge'''
# Check the real problem in the column formatted as dictionary
pprint(sorted(set(df_logs_1['level2dish_coded'].values)))

['{\'dish\' : ["Apples and pears"]}',
 '{\'dish\' : ["Berries / cherries / grapes"]}',
 '{\'dish\' : ["Biscuits with filling"]}',
 '{\'dish\' : ["Biscuits"]}',
 '{\'dish\' : ["Bread/ roll/ baguette", "Cured meats", "Hard cheese"]}',
 '{\'dish\' : ["Bread/ roll/ baguette", "Honey / syrup"]}',
 '{\'dish\' : ["Bread/ roll/ baguette", "Instant noodles"]}',
 '{\'dish\' : ["Bread/ roll/ baguette"]}',
 '{\'dish\' : ["Broccolli, cabbage, cauliflower"]}',
 '{\'dish\' : ["Cake"]}',
 '{\'dish\' : ["Candy"]}',
 '{\'dish\' : ["Chicken", "Other grains"]}',
 '{\'dish\' : ["Chocolate bar"]}',
 '{\'dish\' : ["Chocolate sweets/ snacks"]}',
 '{\'dish\' : ["Cured sausage", "Soft cheese/ cheese spread", "Bread/ roll/ '
 'baguette", "Egg-based dish"]}',
 '{\'dish\' : ["Custards/ sweet puddings"]}',
 '{\'dish\' : ["Dried fruit"]}',
 '{\'dish\' : ["Egg-based dish", "Bread/ roll/ baguette", "Soft cheese/ cheese '
 'spread", "Honey / syrup", "Olives"]}',
 '{\'dish\' : ["Egg-based dish", "Mushroom"]}',
 '{\'dish

In [9]:
# Extract the desired value from the column
df_logs_1 = clean_dict_column(df=df_logs_1, column='level2dish_coded', key='dish')

Step 4: solve problem on column "questions_135633_and_who_are_you_sharing_your_home_with"

In [10]:
from sbees.insecure import clean_braces

# check the real problem in the column
column_135633 = 'questions_135633_and_who_are_you_sharing_your_home_with'
pprint(sorted(set(df_bg_1[column_135633].values)))

['["Partner / spouse", "Children under 18 years old"]',
 '{["Children over 18 years old", "Parents"]}',
 '{["Children over 18 years old", "Partner / spouse"]}',
 '{["Children over 18 years old"]}',
 '{["Children under 18 years old", "Children over 18 years old", "Parents"]}',
 '{["Children under 18 years old", "Partner / spouse"]}',
 '{["Children under 18 years old"]}',
 '{["Friends / Roommates", "Other family members"]}',
 '{["I live alone"],[""]}',
 '{["I live alone"]}',
 '{["Other family members"]}',
 '{["Parents", "Other family members"]}',
 '{["Parents"]}',
 '{["Partner / spouse", "Children 18+"]}',
 '{["Partner / spouse", "Children over 18 years old"]}',
 '{["Partner / spouse", "Children under 18 years old"]}',
 '{["Partner / spouse", "Parents", "Other family members"]}',
 '{["Partner / spouse"],""}',
 '{["Partner / spouse"]}']


In [11]:
# not all values has the brace problem, so the function created handles that
df_bg_1 = clean_braces(df=df_bg_1, column=column_135633)
pprint(sorted(set(df_bg_1[column_135633].values)))

['["Children 18+", "Partner / spouse"]',
 '["Children over 18 years old", "Children under 18 years old", "Parents"]',
 '["Children over 18 years old", "Parents"]',
 '["Children over 18 years old", "Partner / spouse"]',
 '["Children over 18 years old"]',
 '["Children under 18 years old", "Partner / spouse"]',
 '["Children under 18 years old"]',
 '["Friends / Roommates", "Other family members"]',
 '["I live alone"]',
 '["Other family members", "Parents", "Partner / spouse"]',
 '["Other family members", "Parents"]',
 '["Other family members"]',
 '["Parents"]',
 '["Partner / spouse"]']


Step 5: merge DataFrames

PS: the DataFrames were not sorted on merging to keep the supposed order of submission, once project_1 files are supposedly older than project_2 files.

In [12]:
from sbees.secure import concat_dataframes

In [13]:
# the project 2 columns will be used as final column names
map_bg_cols = dict(zip(prj_1_cols, prj_2_cols))
df_logs = concat_dataframes(df1=df_logs_1, df2=df_logs_2)
df_bg = concat_dataframes(df1=df_bg_1, df2=df_bg_2, rename_cols=map_bg_cols)

Step 6: fix genders spelt / capitalized differently

In [14]:
from sbees.secure import clean_columns_values

In [15]:
col_gender = 'questions_135556_what_is_your_gender'
df_bg = clean_columns_values(df=df_bg, columns=[col_gender])

# It shows that "Demale" exists, what is clearly a mistake
set(df_bg[col_gender].values)

{'Demale', 'Female', 'Male', 'Other'}

In [16]:
# Fixing it
map_to_replace = {col_gender: {'Demale': 'Female'}}
df_bg = clean_columns_values(
    df=df_bg,
    columns=[col_gender], 
    replaces=map_to_replace,
)
set(df_bg[col_gender].values)

{'Female', 'Male', 'Other'}

Step 7: set all location names to codes

In [17]:
from sbees.secure import fix_country_columns

In [18]:
# 1º: verify the severity of the problem
sorted(set(df_logs['location_name'].values))

['FR', 'MX', 'South Africa', 'UK', 'US', 'United Kingdom', 'ZA']

In [19]:
# 2ª: fix the problem - small note on conversion UK -> GB as this is the ISO
df_logs = fix_country_columns(df=df_logs, columns=['location_name'])
sorted(set(df_logs['location_name'].values))

['FR', 'GB', 'MX', 'US', 'ZA']

Step 8: merge duplicated columns

In [20]:
from sbees.secure import merge_string_columns

In [21]:
# Verify the existence of duplicated columns on background DataFrame
pprint(sorted(set(df_bg.columns)))

['LSM_group1',
 'questions_134340_what_do_you_do_to_try_to_stay_healthy_tick_all_that_apply',
 'questions_134341_what_drives_you_to_have_a_healthy_lifestyle_you_may_select_up_to_3',
 'questions_134342_do_you_try_to_eat_or_drink_in_any_of_the_following_ways',
 'questions_134344_which_of_the_following_steps_if_any_are_you_taking_to_eat_drink_more_healthily',
 'questions_134347_how_regularly_do_you_exercise_you_can_be_honest_it_s_just_between_us',
 'questions_134348_which_types_of_exercise_do_you_enjoy_doing_tick_all_that_apply',
 'questions_134356_do_you_do_any_of_the_following_to_help_you_sleep_tick_all_that_apply',
 'questions_134357_and_do_you_try_to_avoid_certain_things_in_the_hour_before_you_go_to_sleep_tick_all_that_apply',
 'questions_135556_what_is_your_gender',
 'questions_135557_how_old_are_you',
 'questions_135613_who_do_you_share_your_home_with',
 'questions_224112_149402_210209_including_yourself_how_many_people_do_you_share_your_home_with',
 'user_id']


In [22]:
# and in logs DataFrame
pprint(sorted(set(df_logs.columns)))

['level2dish_coded',
 'location_name',
 'questions_134999_where_are_you_eating_at_the_moment',
 'questions_134999_where_are_you_eating_at_the_moment.1',
 'questions_135000_which_of_the_following_best_describes_where_you_got_your_food_from_today',
 'questions_135004_what_else_if_anything_are_you_doing_while_eating_your_food_for_example_are_you_also_watching_tv_checking_social_media_cooking_etc_coded',
 'questions_135005_who_are_you_with',
 'questions_135019_what_are_the_main_reasons_that_you_chose_this_food_right_now_if_you_are_having_more_than_one_item_please_write_about_the_main_item_you_are_having_please_be_as_detailed_as_you_can_coded',
 'questions_135020_what_other_food_if_any_did_you_consider_having_instead_coded',
 'questions_135022_who_prepared_the_food',
 'questions_223649_149206_210007_finally_which_best_describes_the_food_you_are_having_today',
 'submission_id',
 'submission_timestamp',
 'user_id']


In [23]:
'''This shows that the questions_134999_where_are_you_eating_at_the_moment
column is duplicated on df_logs and will need to be merged, and now it can
be done'''
df_logs = merge_string_columns(
    df=df_logs,
    column_a='questions_134999_where_are_you_eating_at_the_moment',
    column_b='questions_134999_where_are_you_eating_at_the_moment.1',
)

Step 9: merge duplicated values on specific columns

First is necessary to understand which columns will need to be cleaned,
and for that specific case, as it is going to be a massive cleaning,
discover dynamically which columns need it would not be a safe / optimal
choice, so the columns that needs the treatment were verified "simply" by
looking at the data and checking the ones that have an structure of a list.
The columns were then mapped and placed into an asset file.

In [24]:
import json
from sbees.insecure import remove_duplicated_values

In [25]:
with open(f'{assets_folder}list_like_columns_map.json') as file:
    list_like_columns_map = json.load(file)
pprint(list_like_columns_map)

{'background': ['questions_135613_who_do_you_share_your_home_with',
                'questions_134340_what_do_you_do_to_try_to_stay_healthy_tick_all_that_apply',
                'questions_134341_what_drives_you_to_have_a_healthy_lifestyle_you_may_select_up_to_3',
                'questions_134342_do_you_try_to_eat_or_drink_in_any_of_the_following_ways',
                'questions_134344_which_of_the_following_steps_if_any_are_you_taking_to_eat_drink_more_healthily',
                'questions_134348_which_types_of_exercise_do_you_enjoy_doing_tick_all_that_apply',
                'questions_134356_do_you_do_any_of_the_following_to_help_you_sleep_tick_all_that_apply',
                'questions_134357_and_do_you_try_to_avoid_certain_things_in_the_hour_before_you_go_to_sleep_tick_all_that_apply'],
 'logs': ['questions_135005_who_are_you_with',
          'questions_135004_what_else_if_anything_are_you_doing_while_eating_your_food_for_example_are_you_also_watching_tv_checking_social_media_

In [26]:
df_bg = remove_duplicated_values(
    df=df_bg,
    list_like_columns=list_like_columns_map['background'],
)

df_logs = remove_duplicated_values(
    df=df_logs,
    list_like_columns=list_like_columns_map['logs'],
)

Step 10: save the DataFrames

In [27]:
# Could be also saved as XLSX, Parquet, etc.
df_bg.to_csv('output/background_dataset.csv')
df_logs.to_csv('output/logs_dataset.csv')

All done :)